# Fibonacci Sequence:


In mathematics, the Fibonacci numbers, commonly denoted $F_n$ form a sequence, called the Fibonacci sequence, such that each number is the sum of the two preceding ones, starting from 0 and 1. That is:


$ F_n = F_{n - 1} + F_{n - 2} $

In [1]:
# Simple function that computes Fibonacci numbers

function fib(n)
    nums = ones(Int, n)
    for i = 3:n
        nums[i] = nums[i - 1] + nums[i - 2]
    end 
    return nums[n]
end     

fib (generic function with 1 method)

In [2]:
fib(30)

832040

In [3]:
function qsort(a, lo, hi)
    i = lo
    j = hi
    while i < hi
        pivot = a[div(lo + hi, 2)]
        while i <= j
            while a[i] < pivot
                i += 1
            end
            while a[j] > pivot
                j -= 1
            end
            if i <= j
                a[i], a[j] = a[j], a[i]
                i += 1
                j -= 1
            end 
        end
        if lo < j
            qsort(a, lo, j)
        end
        lo = i
        j = hi
    end 
    return a
end 

qsort (generic function with 1 method)

In [4]:
# quick sort in Julia

n = 1_000_000
x = rand(n)

@time x2 = qsort(x, 1, n);

  0.084202 seconds (10.59 k allocations: 531.822 KiB)


In [ ]:
# Parallel processing with @distributed 
using Distributed 

addprocs(5)          

function monte_carlo_pi(n_iter)
    num_inside = @distributed (+) for i = 1:n_iter
        Int(rand()^2 + rand()^2 ≤ 1)
    end
    res = num_inside/(n_iter/4)
    return res
end 

In [ ]:
@time monte_carlo_pi(100_000_000)

In [ ]:
n = 100_000_000
v = SharedArray(randn(n))

@time shr_squeeze_range(v);

### Using Threads for Parallelism

In [ ]:
# NOTE: JULIA_NUM_THREADS env variable is set in .bash_profile
using Base.Threads

function thr_squeeze_range!(x)
    n = length(x)
    @threads for i = 1:n
        if !(-0.5 ≤ x[i] ≤ 0.5) 
            x[i] = 0.5 * sign(x[i])
        end 
    end 
end

In [ ]:
n = 100_000_000
v = randn(n)

@time thr_squeeze_range!(v)

# <center> Julia's Type System </center>

## Julia's Type System 
1. Dynamic
2. Parametric
3. Nominative

## Julia Types:
1. Every object has a type (e.g., `Int`, `Float64`, `Dict`, `Function`)
2. User-defined types behave just like base language types
3. Types (and functions) are parametric 
4. Type annotation can (optionally) be added and can improve performance

### Illustrating Multiple Dispatch

In [ ]:
function foo(a::Int, b::Int) 
    println("I'm adding ints. Yay!!")
    res = a + b
    return res
end 

In [ ]:
foo(3, 1)

In [ ]:
# Adding method to foo()

function foo(a::Float64, b::Int)
    println("Now I'm adding ints and floats!")
    res = a + b
    return res 
end

In [ ]:
methods(+)

### Using Concrete Types for Performance

In [ ]:
function count_vals(v)
    n = length(v)
    cnts = Dict()
    for i = 1:n
        cnts[v[i]] = get(cnts,v[i], 0) + 1
    end
    return cnts
end

In [ ]:
x = rand(['a', 't', 'g', 'c', 1, 3, 7], 100_000_000)

@time res = count_vals(x)

In [ ]:
function count_vals2(v)
    n = length(v)
    cnts = Dict{Char, Int}()
    for i = 1:n
        cnts[v[i]] = get(cnts,v[i], 0) + 1
    end
    return cnts
end

In [ ]:
x = rand(['a', 't', 'g', 'c'], 100_000_000)

@time res = count_vals2(x)

In [ ]:
function count_vals2(X::Matrix)
    n, p = size(X)
    cnts = Dict{Char, Int}()
    for j = 1:p 
        for i = 1:n
            cnts = get(cnts, X[i, j], 0) + 1
        end
    end
    return cnts 
end 

A = rand(['a', 't', 'g', 'c'], 1000, 1000)

count_vals(A) 

## User-Defined Types 

1. User can define composite types (i.e., `struct`s)
2. A `struct` can be mutable or immutable
3. Immutable `struct`s are allocated on stack, mutable are on the heap

In [ ]:
struct Dog
    name 
    age 
    words
end 


struct Cat
    name
    age
    words
end 


In [ ]:

function say_name(x::Dog)
    s = "$(x.words), my name is $(x.name), and I'm a dog."
    println(s)
end

willie = Dog("William Lee", 11, "Woof")    

say_name(willie)

In [ ]:
function say_name(x::Cat)
    for i = 1:rand(1:10)
        print("$(x.words) ")
    end
end

ricky = Cat("Richard", 7, "Meow")

say_name(ricky)

In [ ]:
# Example of multiple dispatch

function say_name(x::Dog, y::Cat)
    println("Two animals fight briefly...\n\nAnd the winner is: $(x.name)")
end 

In [ ]:
say_name(willie, ricky)

In [ ]:
# Define new struct that is Dog/Cat hybrid

struct Dat 
    name
    age
    words
end 

### Operator Overloading

+ Can overload base language's operators (e.g., `+`, `*`, `/`)

In [ ]:
import Base.+ 

function +(x::Dog, y::Cat)
    res = Dat(x.name, y.age, string(x.words, y.words))
    return res 
end


In [ ]:
# Test our new method for +() function

willie + ricky

# <center> Profiling and Introspection </center>

1. Profiling Tools
  * `@time` macro
  * BenchmarkTools.jl package
2. Introspection
  * Type hierarchy 
  * `typeof()`
  * `eltype()`
  * `@code_warntype` 
  * `@code_native`

In [ ]:
# Illustrating type hierarchy 

Int32 <: Float64

In [ ]:
BigInt <: Real

In [ ]:
Bool <: Int64

In [ ]:
Float64 <: AbstractFloat

In [ ]:
a = zeros(100)

typeof(a)

In [ ]:
A = ones(100, 100)

typeof(A)

In [ ]:
# Get type of elements in containers

b = randn(1000)

eltype(b)

In [ ]:
# Get fields of a struct

fieldnames(Dog) 

In [ ]:
function count_vals(v)
    n = length(v)
    cnts = Dict()
    for i = 1:n
        cnts[v[i]] = get(cnts,v[i], 0) + 1
    end
    return cnts
end

x = rand(['a', 't', 'g', 'c'], 100_000)

@code_warntype count_vals(x)

In [ ]:
z = rand(['a', 't', 'g', 'c'], 100_000)

@code_warntype count_vals2(z)

# <center>Integration with other Languages</center>

## Julia and other Languages

Julia has zero-cost interfaces with many technical computing languages:  
1. PyCall.jl
2. RCall.jl
3. `ccall()`
4. Cxx.jl
5. Matlab.jl


### <center>Examples using RCall.jl and Cxx.jl</center>

# <center> Tooling and Package Ecosystem </center>

1. Package manager
  * GitHub
  * Packages installed directly from Julia session using `Pkg.add("<PACKAGE-NAME>")` 
2. Editors and IDEs
  * Atom (Juno plugin)
  * VS Code
  * Emacs (ESS plugin)
  * Jupyter
  * Jupyter Lab

# <center> Challenges </center>

1. Julia is still young
  * Package ecosystem is still growing
  * StackOverflow answers are a bit sparse
  * Breaking changes still occur (mostly in packages, not base language)